# Try to read a vtk file and a numpy array file of the same data and check that they are the same.

In [1]:
import numpy as np
import vtk
from vtk.util.numpy_support import vtk_to_numpy
import read_rectilinear_vtk as rrv
reload(rrv)

<module 'read_rectilinear_vtk' from 'read_rectilinear_vtk.pyc'>

In [171]:
test_file_vtk = ("/home/jensv/rsx/jens_analysis/writing_to_vtk/"
                 "output/2017-01-16-18-15/Bdot_triple_probe_quantities0000.vtk")
test_file_npz = ("/home/jensv/rsx/jens_analysis/writing_to_vtk/"
                 "output/2017-01-20-21-03/Bdot_triple_probe_quantities0000.npz")

In [172]:
vtk_mesh, shape = rrv.get_mesh(vtk_data)
print shape

(30, 38, 162)


In [173]:
vtk_data = rrv.get_data(test_file_vtk)
vtk_mesh, shape = rrv.get_mesh(vtk_data)
(b_x_from_vtk,
 b_y_from_vtk,
 b_z_from_vtk) = rrv.get_vector(vtk_data, shape, 
                                'B_x', 'B_y', 'B_z')
vtk_b_field = [b_x_from_vtk, b_y_from_vtk, b_z_from_vtk]

In [174]:
npz_file = np.load(test_file_npz)
npz_mesh = npz_file['mesh']
npz_b_field = npz_file['b_field']

In [178]:
np.allclose(npz_b_field[0], vtk_b_field[0])

True

# Make test vtk file

Below I made a test vtk file I plotted with seaborn and opened in Visit to make sure the axis swapping that I am doing is really that right thing.
The visual inspection in Visit shows that the data is disaplyed with the correct axis ordering. 

In [153]:
import matplotlib.pyplot as plt
import sys
sys.path.append('../../writing_to_vtk/source/')
import write_canonical_flux_tube_quantities as write
import seaborn as sns

In [162]:
x = 5 
y = 6
z = 7 
x_line = np.arange(x)
y_line = np.arange(y)
z_line = np.arange(z)
mesh = np.meshgrid(x_line, y_line, z_line)
test_field = np.arange(0, 5*6*7)
test_field_numpy = np.reshape(test_field, (y, x, z)) 

In [163]:
sns.heatmap(test_field_numpy[:,:,0], annot=True)
plt.show()

In [164]:
test_field_numpy.shape

(6, 5, 7)

In [165]:
test_field_numpy[0, :, 0]

array([ 0,  7, 14, 21, 28])

In [167]:
x, y, z, variables = write.prepare_for_rectilinear_grid(mesh, (test_field_numpy,), ('test',))
write.write_fields_and_currents_to_structured_mesh('test', 'test', x, y, z, variables, 0)

names length: 1
quantities length: 1
../output/test/test0000


# Read grid

In [7]:
current_grid_file = '/home/jensv/rsx/jens_analysis/output/canonical_quantities/2017-02-22-11-01/canonical_quantities0000.vtk'

In [9]:
vtk_mesh, b_x = rrv.read_scalar(current_grid_file, 'B_x')
print vtk_mesh[0][0, 0 ,0], vtk_mesh[0][0, -1, 0]
print vtk_mesh[1][0, 0, 0], vtk_mesh[1][-1, 0, 0]
print vtk_mesh[2][0, 0, 0], vtk_mesh[2][0, 0, -1]

-0.0219999998808 0.0240000002086
-0.0169189181179 0.0179999992251
0.248999997973 0.416000008583
